# Cleaning Data Exercises

In this exercise, we'll be returning to the American Community Survey data we used previously to measuring racial income inequality in the United States. In today's exercise, we'll be using it to measure the returns to education and how those returns vary by race and gender.




## Gradescope Autograding

Please follow [all standard guidance](https://www.practicaldatascience.org/html/autograder_guidelines.html) for submitting this assignment to the Gradescope autograder, including storing your solutions in a dictionary called `results` and ensuring your notebook runs from the start to completion without any errors.

For this assignment, please name your file `exercise_missing.ipynb` before uploading.

You can check that you have answers for all questions in your `results` dictionary with this code:

```python
assert set(results.keys()) == {
    "ex5_age_young",
    "ex5_age_old",
    "ex7_avg_age",
    "ex8_avg_age",
    "ex9_num_college",
    "ex11_share_male_w_degrees",
    "ex11_share_female_w_degrees",
    "ex12_comparing",
}
```


### Submission Limits

Please remember that you are **only allowed three submissions to the autograder.** Your last submission (if you submit 3 or fewer times), or your third submission (if you submit more than 3 times) will determine your grade Submissions that error out will **not** count against this total.

## Exercises

### Exercise 1

For these cleaning exercises, we'll return to the ACS data we've used before one last time. We'll be working with `US_ACS_2017_10pct_sample.dta`. Import the data (please use url for the autograder).

In [1]:
import pandas as pd

pd.set_option("mode.copy_on_write", True)


# loading the dataset
acs = pd.read_stata(
    "https://github.com/nickeubank/MIDS_Data/raw/master/US_AmericanCommunitySurvey/US_ACS_2017_10pct_sample.dta"
)

# initializing dictionary
results = {}

### Exercise 2

For our exercises today, we'll focus on `age`, `sex`, `educ` (education), and `inctot` (total income). Subset your data to those variables, and quickly look at a sample of 10 rows.

In [2]:
acs = acs[["age", "educ", "inctot", "sex"]]
print("The following is a table that is subsetted for age, education, income, and sex")
print(acs)

The following is a table that is subsetted for age, education, income, and sex
       age                       educ   inctot     sex
0        4  nursery school to grade 4  9999999  female
1       17                   grade 11     6000  female
2       63         4 years of college     6150    male
3       66                   grade 12    14000  female
4        1        n/a or no schooling  9999999    male
...     ..                        ...      ...     ...
318999  33         4 years of college    22130  female
319000   4  nursery school to grade 4  9999999  female
319001  20                   grade 12     5000    male
319002  47        5+ years of college   240000    male
319003  33        5+ years of college    48000    male

[319004 rows x 4 columns]


### Exercise 3

As before, all the values of `9999999` have the potential to cause us real problems, so replace all the values of `inctot` that are `9999999` with `np.nan`. 

In [3]:
# replace all the values of `inctot` that are `9999999` with `np.nan`
import numpy as np

acs.loc[acs["inctot"] == 9999999, "inctot"] = np.nan
acs["inctot"].value_counts()
np.random.seed(12)
acs.sample(20)

,age,educ,inctot,sex
136741,15,grade 9,0.0,male
40068,77,grade 12,36200.0,male
59267,58,4 years of college,0.0,female
141069,59,grade 12,8400.0,male
212024,56,grade 12,100000.0,male
184478,56,4 years of college,80000.0,female
112077,48,5+ years of college,175000.0,male
263712,58,grade 12,9100.0,female
77546,22,grade 12,0.0,female
107543,60,4 years of college,61200.0,male


### Exercise 4

Attempt to calculate the average age of people in our data. What do you get? Why are you getting that error?

You *should* get an error in trying to answer this question, but **PLEASE LEAVE THE CODE THAT GENERATES THIS ERROR COMMENTED OUT SO YOUR NOTEBOOK WILL RUN IN THE AUTOGRADER**. 

Then talk about the error in a markdown cell.

In [4]:
# I am now going to be calculating the average age of people in our data
# acs["age"].mean()

#### Exercise 4: The reason why there is an error is because age is not appearing as a numerical variable, it is coming as a categorical variable. You can't take the mean of a categorical variable. 

### Exercise 5

We want to be able to calculate things using age, so we need it to be a numeric type. Check the current type of `age`, and look at all the values of `age` to figure out why it's categorical and not numeric. You should find two problematic categories. Store the values of these categories in `"ex5_age_young"` and `"ex5_age_old"` (once you find them, it should be clear which is which).

In [5]:
# print(acs["age"].dtype)

# print(acs.loc[:,"age"])

# acs.loc[acs["age"] == "less than 1 year old", 'age']

# acs[~acs["age"].str.isnumeric()]

results["ex5_age_young"] = "less than 1 year old"
results["ex5_age_old"] = "90 (90+ in 1980 and 1990)"


print("The dictionary now has the string attached to different ages")
results

The dictionary now has the string attached to different ages


{'ex5_age_young': 'less than 1 year old',
 'ex5_age_old': '90 (90+ in 1980 and 1990)'}

### Exercise 6

In order to convert `age` into a numeric variable, we need to replace those problematic entries with values that `pandas` can later convert into numbers. Pick appropriate substitutions for the existing values and replace the current values. 

**Hint 1:** Categorical variables act like strings, so you might want to use string methods! 

**Hint 2:** Remember that characters like parentheses, pluses, asterisks, etc. are special in Python strings, and you have to escape them if you want them to be interpreted literally!

**Hint 3:** Because the US Census has been conducted regularly for hundreds of years but exactly how the census has been conducted have occasionally changed, variables are sometimes coded in a way that might be interpreted in different ways for different census years. For example, hypothetically, one might write `90 (90+ in 1980 and 1990)` if the Censuses conducted in 1980 and 1990 used to top-code age at 90 (any values *over* 90 were just coded as 90), but more recent Censuses no longer top-coded age and recorded ages over 90 as the respondents actual age.

In [6]:
# Exercise 6

acs["age"] = acs["age"].str.replace(results["ex5_age_young"], "0")
acs["age"] = acs["age"].str.replace("90 \(90\+ in 1980 and 1990\)", "90", regex=True)

acs["age"] = acs["age"].astype(int)

print("The age is now numerical from categorical")

The age is now numerical from categorical


### Exercise 7

Now convert age from a categorical to numeric. Calculate the average age amoung this group, and store it in `"ex7_avg_age"`.

In [7]:
print(acs["age"].mean())

results["ex7_avg_age"] = acs["age"].mean()
print("The aveage age among this group is {:.2f}".format(results["ex7_avg_age"]))

41.30384885455982
The aveage age among this group is 41.30


### Exercise 8

Let's now filter out anyone in our data whose age is less than 18. Note that before made `age` a numeric variable, we couldn't do this! Again, calculate the average age and this time store it in `"ex8_avg_age"`. 

Use this sample of people 18 and over for all subsequent exercises.

In [8]:
# Execercise 8 where we are trying to calculate the average age of people under 18
age_filtered = acs[acs["age"] >= 18]
results["ex8_avg_age"] = age_filtered["age"].mean()

print(
    "The average age of people who are over 18 is {:.0f} years old".format(
        results["ex8_avg_age"]
    )
)

The average age of people who are over 18 is 50 years old


### Exercise 9

Create an indicator variable for whether each person has *at least* a college Bachelor's degree called `college_degree`. Use this variable to calculate the number of people in the dataset with a college degree. You may assume that to get a college degree you need to complete at least 4 years of college. Save the result as `"ex9_num_college"`.

In [9]:
# Exercise 9 creating an indicator variable  to see how many students completed at least 4 years of college
college_degree = ["4 years of college", "5+ years of college"]

results["ex9_num_college"] = (
    age_filtered.loc[age_filtered["educ"].isin(college_degree)].value_counts().sum()
)

print(
    "This is the number of students who completed at least 4 years of college {:.0f}".format(
        results["ex9_num_college"]
    )
)

This is the number of students who completed at least 4 years of college 77013


### Exercise 10

Let's examine how the educational gender gap. Use `pd.crosstab` to create a cross-tabulation of `sex` and `college_degree`. `pd.crosstab` will give you the number of people who have each combination of `sex` and `college_degree` (so in this case, it will give us a 2x2 table with Male and Female as rows, and `college_degree` True and False as columns, or vice versa. 

In [10]:
# This is a 2 x 2 table with Male and Females as rows and college degree True or False as columns

gender = age_filtered["sex"]

degree = age_filtered["educ"].isin(college_degree)


cross_tab = pd.crosstab(gender, degree)
print(cross_tab)

educ    False  True 
sex                 
male    85821  36181
female  90200  40832


### Exercise 11

Counts are kind of hard to interpret. `pd.crosstab` can also normalize values to give percentages. Look at the `pd.crosstab` help file to figure out how to normalize the values in the table. Normalize them so that you get the share of men with and without college degree, and the share of women with and without college degrees.

Store the share (between 0 and 1) of men with college degrees in `"ex11_share_male_w_degrees"`, and the share of women with degrees in `"ex11_share_female_w_degrees"`.

In [11]:
# Exercise 11 normalize them so you can get share of men/women with and without college degrees
cross_tab_normalized = pd.crosstab(gender, degree, normalize="index")
print(cross_tab_normalized)

results["ex11_share_male_w_degrees"] = cross_tab_normalized.loc["male", True]
results["ex11_share_female_w_degrees"] = cross_tab_normalized.loc["female", True]
print(
    "The proportion of females with degrees is {:.2f} and the number of males with degrees is {:.2f}".format(
        results["ex11_share_female_w_degrees"], results["ex11_share_male_w_degrees"]
    )
)

educ       False     True 
sex                       
male    0.703439  0.296561
female  0.688381  0.311619
The proportion of females with degrees is 0.31 and the number of males with degrees is 0.30


### Exercise 12

Now, let's recreate that table for people who are 40 and over and people under 40. Over time, what does this suggest about the absolute difference in the share of men and women earning college degrees? Has it gotten larger, stayed the same, or gotten smaller? Store your answer (either `"the absolute difference has increased"` or `"the absolute difference has decreased"`) in `"ex12_comparing"`.

In [12]:
# Exercise 12 to create a table for people who are 40/over and people who are under 40
age_40 = acs["age"] < 40
cross_age_40_tab = pd.crosstab(
    gender, degree, values=age_40, aggfunc="sum", normalize="all"
)

print("True if they are 40 and below and False if they are 40 and above")
print(cross_age_40_tab)

results["ex12_comparing"] = "the absolute difference has increased"
results

True if they are 40 and below and False if they are 40 and above
educ       False     True 
sex                       
male    0.373499  0.129095
female  0.331128  0.166278


{'ex5_age_young': 'less than 1 year old',
 'ex5_age_old': '90 (90+ in 1980 and 1990)',
 'ex7_avg_age': 41.30384885455982,
 'ex8_avg_age': 49.75769659413359,
 'ex9_num_college': 77013,
 'ex11_share_male_w_degrees': 0.29656071211947344,
 'ex11_share_female_w_degrees': 0.3116185359301545,
 'ex12_comparing': 'the absolute difference has increased'}

### Exercise 13

In words, what is causing the change noted in Exercise 12 (i.e., looking at the tables above, tell me a story about Men and Women's College attainment).

#### The reason why women are catching up to men when we look at younger ages over 18 is due to the how society has changed the view on a women's role and women's rights. Now it is more acceptable and more desirable for a women to get a college degree. This means now women are filling the gap we saw in the cross table of women and men over 40. There is also imporvement on women's rights and education scholarships that are making it more attainable for women to get an education.

## Want More Practice?

Calculate the educational racial gap in the United States for White Americans, Black Americans, Hispanic Americans, and other groups. 

Note that to do these calculations, you'll have to deal with the fact that unlike most Americans, the American Census Bureau treats "Hispanic" not as a racial category, but a linguistic one. As a result, the racial category "White" in `race` actually includes most Hispanic Americans. For this analysis, we wish to work with the mutually exclusive categories of "White, non-Hispanic", "White, Hispanic", "Black (Hispanic or non-Hispanic)", and a category for everyone else. 

In [13]:
assert set(results.keys()) == {
    "ex5_age_young",
    "ex5_age_old",
    "ex7_avg_age",
    "ex8_avg_age",
    "ex9_num_college",
    "ex11_share_male_w_degrees",
    "ex11_share_female_w_degrees",
    "ex12_comparing",
}